#### Agents SDK Course

## Tracing

Agents SDK integrates with OpenAI's built-in **Traces** dashboard found within the OpenAI Platform. In this example, we'll take a look at both the default tracing enabled by default whenever we use Agents SDK (with an OpenAI API key present), and custom tracing.

## Accessing the Traces Dashboard

We should first confirm that we have access to the **Traces** dashboard in the [OpenAI Platform](https://platform.openai.com/). You should be able to find the dashboard like so:

TK add visual steps

By default the traces dashboard is _not_ visible by anyone but the organization owner. So, if you are _not_ the organization owner _or_ you need to provide access to other members of your organization you would do that via the **Data controls** setting, which can be found here:

![Visual steps showing how to set traces visibility for an organization in the OpenAI Platform](../assets/traces-data-control.png)

After gaining access to the dashboard, _if_ you have been following through with other chapters in the course you should see a list of traces already. If you don't — no problem — we'll be creating traces in the following steps of this example.

As usual, we need to set our `OPENAI_API_KEY`, which we find in the [OpenAI Platform](https://platform.openai.com/api-keys).

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
    getpass.getpass("OpenAI API Key: ")

### Tracing Basic Agent Prompting

First we need to create an agent, we will use the `Agent` class to create a basic agent.

In each `Agent` object we have some fundamental parameters, those are:
- `name`: The name of the agent
- `instructions`: The system prompt for the agent
- `model`: The model to use for the agent

The `instructions` parameter is where we will pass our system prompt, this will be used to guide the behavior of the agent


In [2]:
from agents import Agent # object class

prompt_agent = Agent(
    name="Tracing Prompt Agent",  # name of the agent
    instructions="Speak like a pirate.",  # system prompt
    model="gpt-4.1-nano"  # model to use
)

When running this agent we will automatically find traces being sent to our traces dashboard.

In [3]:
from agents import Runner

query = "Write a one-sentence poem"

prompt_result = await Runner.run(
    starting_agent=prompt_agent,  # agent to start the conversation with
    input=query  # input to pass to the agent
)

We should now see a trace in the dashboard with the default workflow name (Agents SDK uses `Agent workflow` for agent runs) and the name of our agent, ie `"Tracing Prompt Agent"`:

![Default trace in Platform](../assets/traces-default-trace.png)

In the trace preview we can see key information such as whether any handoffs occured or tools were used, and how long the workflow took to complete.

We can also click on the trace to see a breakdown of each step in our trace, ie the individual _spans_, and we click on one of those spans to see the _span details_. Including LLM information, input / outputs, and the number of tokens that we have used.

![Default trace span detail in Platform](../assets/traces-default-trace-span.png)

## Building Custom Traces

To customize our tracing, we can use the `trace` function from the Agents SDK library. With this, we can add and/or modify any defined parameters to anything ran within the context of our `trace`.

```python
with trace(...):
    # any agents-sdk code run here will inherit the parameters
    # of our `trace`
    ...
```

There are various parameters we can set, the primary parameters are:

- `workflow_name`: Name given to the workflow, this will be displayed as the primary _name_ of the trace.
- `group_id`: ID given to a group of traces. Throughout this notebook we set this to `Agents SDK Course: Tracing` so we can easily identify traces that have been generated from this notebook.
- `trace_id`: ID of the individual trace (recommended to use `util.gen_trace_id()` if using)
- `metadata`: Dictionary of data given with the workflow data
- `disabled`: If true it will not return a trace

We will try setting just the `workflow_name` and `group_id`.

In [4]:
from agents import Runner, trace # object class

group_id = "Agents SDK Course: Tracing"

with trace(workflow_name="Prompt Agent Workflow", group_id=group_id):
    prompt_result = await Runner.run(
        starting_agent=prompt_agent,  # agent to start interaction with
        input=query # input to pass to the workflow
    )

From this, we should find a new **Prompt Agent Workflow** trace in our dashboard. We can also search for it by entering `Agents SDK Course: Tracing` in the _Group_ search bar.

![Filtering by the group field to find our new prompt agent workflow trace](../assets/traces-custom-trace.png)

### Tracing Agent Tools

Again, we need to create an agent.

In this `Agent` object we have an additional parameter, this is:
- `tools`: The tools provided to the agent for usage.

In order to use this we need to aquire a tool to use, we can import `WebSearchTool` from the agents library, this tool will allow the `agent` object to search the web for results.

In [5]:
from agents import Agent, WebSearchTool

tool_agent = Agent(
    name="Tracing Tool Agent",
    instructions=(
        "You are a web search agent that searches the web for information on the "
        "user's queries."
    ),
    tools=[
        WebSearchTool()
    ],
    model='gpt-4.1-nano'
)

This time we can use the trace parameters to define the tool we have given to our agent within the metadata, this will provide better information in the trace.

In [6]:
query = "What are the current headlines in the news?"

with trace(workflow_name="Tool Trace", group_id="Tracing 101", metadata={"Tools": "WebSearchTool"}):
    tool_result = await Runner.run(
        starting_agent=tool_agent,
        input=query
    )

In [22]:
print("Tool Agent Result:", tool_result.final_output)

Tool Agent Result: Here are some of the latest news headlines as of April 24, 2025:

**U.S. News:**

- **Polar Vortex Grips Much of the U.S.:** A severe polar vortex is causing frigid temperatures across large portions of the United States, leading to disruptions and safety concerns. ([apnews.com](https://apnews.com/?utm_source=openai))

- **Minneapolis Agrees to Overhaul Police Training:** In response to the murder of George Floyd, Minneapolis has agreed to significant changes in police training and use-of-force policies. ([apnews.com](https://apnews.com/?utm_source=openai))

**International News:**

- **Strong Earthquake in Western China:** A powerful earthquake has struck western China, resulting in at least 53 fatalities and extensive damage. ([apnews.com](https://apnews.com/?utm_source=openai))

- **Gunmen Attack Bus in West Bank:** An attack on a bus in the occupied West Bank has left three people dead, escalating tensions in the region. ([apnews.com](https://apnews.com/?utm_sour